In [74]:
import mlflow
import numpy as np
from mlflow import MlflowClient
import pandas as pd

MLFLOW_TRACKING_URI = "sqlite:///wine_pred.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [45]:
experiments = client.search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print("-")

Experiment ID: 3
Name: client_file2
Artifact Location: /workspaces/mlops_wine/mlruns/3
-
Experiment ID: 2
Name: client
Artifact Location: /workspaces/mlops_wine/mlruns/2
-
Experiment ID: 1
Name: wine-quality-prediction
Artifact Location: /workspaces/mlops_wine/mlruns/1
-
Experiment ID: 0
Name: Default
Artifact Location: mlflow-artifacts:/0
-


In [12]:
client.create_experiment(name="client_file2")

'3'

In [82]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.accuracy < 1",
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.accuracy DESC"]
)
for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.data.metrics['accuracy']:.2f}")

run id: c85f35d99a4b4feaa21f84899fec0443, accuracy: 0.69
run id: b698158da9154d36b157f1f9a2a04b00, accuracy: 0.66
run id: ef74486a69674befa413a77cccc27080, accuracy: 0.64
run id: d627ff48b5544e7b8a87d2564f806960, accuracy: 0.64
run id: 37a77f6654084abb9d5478bed336be38, accuracy: 0.63
run id: e289edb7fcd54190856cf5870b4bc0ca, accuracy: 0.57
run id: cec9ccc5938a45b1b068857a4f7ef3a8, accuracy: 0.52
run id: fde77d0f244f4eb283a8701f94aff267, accuracy: 0.35


In [83]:
registered_models = client.search_registered_models()

for model in registered_models:
    print(f"Nom du modèle: {model.name}")
    for version in model.latest_versions:
        print(f"  Version: {version.version}, Run ID: {version.run_id}")

Nom du modèle: wine_classifier
  Version: 1, Run ID: c85f35d99a4b4feaa21f84899fec0443
  Version: 2, Run ID: b698158da9154d36b157f1f9a2a04b00
  Version: 3, Run ID: cec9ccc5938a45b1b068857a4f7ef3a8
Nom du modèle: wine_other
  Version: 1, Run ID: ef74486a69674befa413a77cccc27080
  Version: 2, Run ID: d627ff48b5544e7b8a87d2564f806960
Nom du modèle: wine_regressor
  Version: 1, Run ID: fde77d0f244f4eb283a8701f94aff267
  Version: 2, Run ID: 37a77f6654084abb9d5478bed336be38


In [81]:
#model_name = "wine_classifier"
#model_name = "wine_other"
model_name = "wine_regressor"

model_versions = client.search_model_versions(f"name='{model_name}'")

production_model = None
for version in model_versions:
    if version.current_stage == "Production":
        production_model = version
        break

if not production_model:
    raise Exception("Aucune version de modèle en Production trouvée.")

model_uri = production_model.source

local_path = mlflow.artifacts.download_artifacts(model_uri)

model_path = os.path.join(local_path, "model.pkl")
model = joblib.load(model_path)

feature_names = [
    "fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides",
    "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"
]
input_data = pd.DataFrame(np.array([[
    15.0,  
    1.58, 
    0.0,   
    65.8, 
    0.611, 
    3.0,   
    125.0, 
    1.003, 
    2.74,  
    2.00,  
    8.4    
]]), columns=feature_names)

prediction = model.predict(input_data)
print(f"Prédiction de la qualité du vin: {prediction[0]}")

Prédiction de la qualité du vin: 4
